In [ ]:
%%capture
!pip install keybert[all]
#!pip install keybert
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate

In [ ]:
!pip install pyarrow
!pip install spacy
!pip install fastparquet

In [ ]:
!pip install auto-gptq # for cuda versions other than 11.7, refer to installation guide in above link
!pip install transformers optimum peft

In [ ]:
import spacy

In [ ]:
import pandas as pd
import pyarrow
import fastparquet
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM, GPTQConfig


ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from keybert import KeyBERT

In [ ]:
parquet_file = '/content/drive/MyDrive/reviews_aws_sample.parquet'
reviews = pd.read_parquet(parquet_file, engine='pyarrow')

In [ ]:
reviews[['review_body']][:10]

reviews[(reviews['product_category'] == 'Watches') & (reviews['product_title']== 'Timex Women\'s Easy Reader Leather Strap Watch')][['review_body']][:10]
product = reviews[(reviews['product_category'] == 'Watches') & (reviews['product_title']== 'Timex Women\'s Easy Reader Leather Strap Watch')]

In [ ]:
prompt1 = " ".join(product['review_body'])

In [ ]:
kw_model = KeyBERT(model="all-mpnet-base-v2")

In [ ]:
kw_model.extract_keywords(prompt1,
                          top_n=6,
                          keyphrase_ngram_range=(1, 1),
                          use_mmr=True,
                          diversity=0.1,
                          #use_maxsum=True,
                          #nr_candidates=20,
                          )

In [ ]:
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("xlm-r-bert-base-nli-stsb-mean-tokens", device="cuda")

In [ ]:
import torch
import transformers


model = transformers.AutoModel.from_pretrained(
    'numind/entity-recognition-general-sota-v1',
    output_hidden_states=True
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    'numind/entity-recognition-general-sota-v1'
)

text = prompt1

encoded_input = tokenizer(
    text,
    return_tensors='pt',
    padding=True,
    truncation=True
)
output = model(**encoded_input)

# for better quality
emb = torch.cat(
    (output.hidden_states[-1], output.hidden_states[-7]),
    dim=2
)

# for better speed
# emb = output.hidden_states[-1]


(…)ognition_NER-v1/resolve/main/config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

(…)ER-v1/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

(…)cognition_NER-v1/resolve/main/vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

(…)cognition_NER-v1/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)ition_NER-v1/resolve/main/tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

(…)-v1/resolve/main/special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

In [ ]:
print(output)

### Segunda Parte

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta")

In [ ]:

model = AutoModelForCausalLM.from_pretrained("distilgpt2")
model.generation_config


GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256
}

In [ ]:
pipe = pipeline("text-generation", model="HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.bfloat16, device_map="auto")

# We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
messages = [
    {
        "role": "system",
        "content": "You are bot that makes short max 200 words reviews from keywords that are given from the user, for a watch",
    },
    {"role": "user", "content": "affordable, attractive, easy, loved, nice"},
]
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])